In [21]:
import re

def make_ingredient_dict(ingredient_list):
    """
    재료 리스트를 {'재료명': '양/단위'} 형태의 딕셔너리로 변환.
    연속된 공백(2칸 이상)을 기준으로 재료명과 단위를 구분하며,
    숫자나 '약간', '적당히', '손가락 길이' 등 표현도 단위로 포함.
    """
    ingredient_dict = {}

    # 단위로 간주할 표현들
    amount_keywords = [
        '약간', '적당히', '조금', '큼직하게', '솔솔', '톡톡', '적당량', '한줌', '한스푼',
        '보통사이즈', '작은거', '작', '큰캔', '선택사항', '생략가능', '크게', '깍아서',
        '보통', '보통크기', '중간크기', '중간사이즈', '또는', '손가락길이', '손가락 길이',
        '길이', '정도', '약', '소량', '듬뿍', '넉넉히', '취향껏', '원하는만큼', '맘껏', '기호에맞게', '필요한만큼',
        '적당량으로', '한줌정도', '중간거', '많이', '큰거', '살짝', '넉넉히', '채썬것', 
        '중간것', '큰것', '팍팍', 'or', '선택', '기호에따라', '기호에따라서', '작은거'
        
    ]

    # 숫자 or 키워드 기반 단위 패턴
    amount_pattern = re.compile(
        r'('
        r'(?:\d[\d\/\.\~]*\s*[가-힣A-Za-z]+)'  # ex: 1/2컵, 2큰술
        + '|' + '|'.join(amount_keywords) +     # ex: 약간, 손가락 길이
        r')'
    )

    for item in ingredient_list:
        if not item or not isinstance(item, str):
            continue

        # 1️⃣ 연속된 공백 기준으로 split 시도
        parts = re.split(r'\s{2,}', item.strip())
        if len(parts) == 2:
            name, raw_amount = parts[0].strip(), parts[1].strip()
        else:
            # 연속 공백이 없는 경우, 기존 로직으로 처리
            parts = re.split(
                r'\s*(?=\d|'
                + '|'.join(amount_keywords)
                + r')', item
            )
            name = parts[0].strip()
            matches = amount_pattern.findall(item)
            raw_amount = ' '.join(matches).strip() if matches else ""

        # 2️⃣ 불필요 문자 제거
        name = re.sub(r'[^가-힣A-Za-z0-9\s]', '', name).strip()
        raw_amount = raw_amount.replace('  ', ' ').strip()

        if name:
            ingredient_dict[name] = raw_amount or None

    return ingredient_dict


In [ ]:
import pandas as pd
import ast

# ===============================
# 2️⃣ CSV 불러오기 + 변환
# ===============================
df = pd.read_csv("recipe_by_type.csv")

processed_ingredients = []

for idx, row in df.iterrows():
    try:
        ingredient_list = ast.literal_eval(row['ingredient_full'])
        cleaned_dict = make_ingredient_dict(ingredient_list)
        processed_ingredients.append(cleaned_dict)
    except Exception as e:
        print(f"❌ {row.get('recipe_nm_ko', 'Unknown')} 처리 중 오류: {e}")
        processed_ingredients.append(None)

# 🔹 순서 그대로 (ID나 index 순서 유지)
df['ingredient_full'] = processed_ingredients

# 🔹 저장
df.to_csv("recipe_by_type_cleaned.csv", index=False, encoding="utf-8-sig")
print("순서 그대로 각 레시피에 재료가 정확히 매칭되었습니다!")

✅ 순서 그대로 각 레시피에 재료가 정확히 매칭되었습니다!


-----------------------------------------------------------

In [ ]:
# -------------------------------
# ✅ 2️⃣ 레시피 단건 처리 함수, 크롤링 해올떄 사용할 수 있는 함수 
# -------------------------------
def process_single_recipe(recipe_name, ingredient_full_str):
    """
    크롤링 중 레시피 1개씩 들어올 때
    ingredient_full을 dict로 변환해주는 함수
    """
    try:
        ingredient_list = ast.literal_eval(ingredient_full_str)
        cleaned_dict = make_ingredient_dict(ingredient_list)
        print(f"✅ '{recipe_name}' 재료 정제 완료")
        return cleaned_dict
    except Exception as e:
        print(f"❌ '{recipe_name}' 처리 오류: {e}")
        return None

In [ ]:
# 크롤링 도중 
recipe_name = "소세지강정"
ingredient_str = "['비엔나 소세지 30개', '양파 1개', '캐첩 5큰술', '간장 1스푼']"

cleaned_ingredient = process_single_recipe(recipe_name, ingredient_str)
print(cleaned_ingredient)


In [ ]:
# or 크롤링 결과를 df로 쌓을떄 
import pandas as pd

recipes = []

for recipe in crawled_data:  # 크롤링 루프 안에서
    cleaned_ing = process_single_recipe(recipe['name'], recipe['ingredient_full'])
    recipes.append({
        "recipe_id": recipe['id'],
        "recipe_nm_ko": recipe['name'],
        "ingredient_full": cleaned_ing,  # dict 저장
        "step_text": recipe['steps']
    })

df = pd.DataFrame(recipes)
df.to_csv("recipe_crawled_cleaned.csv", index=False, encoding="utf-8-sig")
